In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=a8497ecd1f97be818b1ff463452c1fae75462e5b39b5da3bfc0f9b6c91e641a5
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0256 ,random_state=0 , l1_ratio=0.797, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 6.18 , C = 938.57 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_242000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_24_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=871937, Mon Apr 13 14:49:17 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 4.647342854770534e+03 1.0e+00 2.57e-01  3e-01  3e-01 0:00.0
    2     30 4.647342720574976e+03 1.0e+00 2.50e-01  2e-01  3e-01 0:00.0
    3     45 4.647342793649037e+03 1.1e+00 2.46e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4647.34324818] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4647.34379319] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 3.210911867376171e+03 1.5e+00 7.62e-02  7e-02  8e-02 0:02.0
  200   3000 2.717211145943859e+03 1.6e+00 4.87e-03  4e-03  5e-03 0:03.9
  300   4500 2.715183418077729e+03 1.6e+00 3.30e-04  3e-04  3e-04 0:05.9
  400   6000 2.715174435773029e+03 1.6e+00 2.39e-05  2e-05  2e-05 0:07.8
  500   7500 2.715174386093827e+03 1.6e+00 1.42e-06  1e-06  1e-06 0:09.7
  600   9000 2.715174385942035e+03 1.6e+00 1.02e-07  8e-08  9e-08 0:11.7
  700  10500 2.715174385941114e+03 1.6e+00 7.08e-09  5e-09  6e-09 0:13.6
  789  11835 2.715174385941112e+03 1.8e+00 1.81e-09  1e-09  2e-09 0:15.4
termination on tolfun=1e-11 (Mon Apr 13 14:49:35 2020)
final/bestever f-value = 2.715174e+03 2.715174e+03
incumbent solution: [ 0.04675072  0.02542598 -0.14006052 -0.03997052 -0.02560333  0.01900346
 -0.06408277 -0.04565247 ...]
std deviations: [1.42139519e-09 1.33855964e-09 1.44227583e-09 1.39817452e-09
 1.35817576e-09 1.49761299e-09 1.32030683e-09 1.43630755e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4649.69713495] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4649.69713495] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 3.905303123849475e+03 1.4e+00 2.48e-02  2e-02  2e-02 0:01.4
  200   3000 3.859663285907642e+03 1.4e+00 1.48e-03  1e-03  1e-03 0:02.7
  300   4500 3.859475157524766e+03 1.5e+00 7.03e-05  6e-05  7e-05 0:04.1
  400   6000 3.859474783173079e+03 1.5e+00 4.08e-06  3e-06  4e-06 0:05.5
  500   7500 3.859474782129350e+03 1.5e+00 1.84e-07  1e-07  2e-07 0:06.8
  600   9000 3.859474782127299e+03 1.6e+00 1.02e-08  8e-09  9e-09 0:08.2
  700  10500 3.859474782127291e+03 1.8e+00 2.77e-09  2e-09  2e-09 0:09.6
  701  10515 3.859474782127292e+03 1.8e+00 2.79e-09  2e-09  2e-09 0:09.6
termination on tolfun=1e-11 (Mon Apr 13 14:49:47 2020)
final/bestever f-value = 3.859475e+03 3.859475e+03
incumbent solution: [-0.20965541  0.23430562 -0.18941671  0.19926813  0.16801467 -0.18444434
  0.18802589  0.13200742 ...]
std deviations: [2.09622682e-09 2.06442764e-09 2.26224706e-09 2.10503939e-09
 2.10038134e-09 2.21584070e-09 2.17043721e-09 2.14409760e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

...

  100   1500 2.875102041738263e+03 1.5e+00 4.08e-02  4e-02  4e-02 0:02.0
  200   3000 2.715908091709638e+03 1.5e+00 3.05e-03  3e-03  3e-03 0:03.9
  300   4500 2.715177662454295e+03 1.6e+00 1.90e-04  2e-04  2e-04 0:05.9
  400   6000 2.715174395748747e+03 1.6e+00 1.11e-05  9e-06  1e-05 0:07.8
  500   7500 2.715174385986147e+03 1.7e+00 7.47e-07  6e-07  7e-07 0:09.7
  600   9000 2.715174385941283e+03 1.7e+00 5.16e-08  4e-08  5e-08 0:11.7
  700  10500 2.715174385941112e+03 1.7e+00 4.90e-09  4e-09  4e-09 0:13.7
  777  11655 2.715174385941111e+03 1.7e+00 1.88e-09  1e-09  2e-09 0:15.2
termination on tolfun=1e-11 (Mon Apr 13 15:05:01 2020)
final/bestever f-value = 2.715174e+03 2.715174e+03
incumbent solution: [ 0.04675072  0.02542598 -0.14006053 -0.03997052 -0.02560332  0.01900346
 -0.06408277 -0.04565247 ...]
std deviations: [1.40195580e-09 1.48289985e-09 1.51324630e-09 1.49319345e-09
 1.37960303e-09 1.54808150e-09 1.42167997e-09 1.42332417e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4649.69713492] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4649.69712905] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 4.015594410928207e+03 1.4e+00 2.04e-02  2e-02  2e-02 0:01.4
  200   3000 3.994881552037458e+03 1.4e+00 9.57e-04  9e-04  9e-04 0:02.7
  300   4500 3.994836256012446e+03 1.5e+00 6.21e-05  5e-05  6e-05 0:04.1
  400   6000 3.994836069527981e+03 1.5e+00 4.26e-06  4e-06  4e-06 0:05.4
  500   7500 3.994836068453372e+03 1.6e+00 2.63e-07  2e-07  2e-07 0:06.8
  600   9000 3.994836068450319e+03 1.6e+00 1.57e-08  1e-08  1e-08 0:08.2
  700  10500 3.994836068450308e+03 1.8e+00 3.43e-09  3e-09  3e-09 0:09.6
  717  10755 3.994836068450308e+03 1.8e+00 3.08e-09  2e-09  3e-09 0:09.8
termination on tolfun=1e-11 (Mon Apr 13 15:05:14 2020)
final/bestever f-value = 3.994836e+03 3.994836e+03
incumbent solution: [ 0.41603757  0.39726236 -0.13214832  0.44279941  0.40479205  0.45548759
  0.11560693  0.05886012 ...]
std deviations: [2.47986448e-09 2.31677935e-09 2.41578898e-09 2.40373814e-09
 2.41988584e-09 2.52502523e-09 2.39732122e-09 2.47524358e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6169.57622327] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6632.56709326] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -2.840793216124743e+07 2.0e+00 1.99e+01  2e+01  2e+01 0:01.9
  139   2085 -4.329552484483438e+09 2.1e+00 2.47e+02  2e+02  3e+02 0:02.6
termination on tolfacupx=1000.0 (Mon Apr 13 15:05:17 2020)
final/bestever f-value = -4.423128e+09 -4.423128e+09
incumbent solution: [  360.22070423   516.71843691  1202.76269073  1541.57094756
 -2232.93867744  -540.89696903   228.40956139  -852.06913951 ...]
std deviations: [238.72864708 235.57151769 237.2148183  245.49390555 248.80183147
 237.43433607 233.5883459  242.40440572 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F24_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:50])
    SVM_Fun [i] = F24(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F24(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([860.19646685]), count=array([1])) 24.812353524486937
SVM
ModeResult(mode=array([809.96855958]), count=array([1])) 22.55982734079779
ELN
ModeResult(mode=array([2.93947147e+11]), count=array([1])) 65546515269.58327
